In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.8/702.8 kB 3.9 MB/s eta 0:00:00


In [ ]:

## importing required libraries
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

Connecting to the google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing libraries,
Direction to paths,
Splitting 80% for training and 20% for testing

In [ ]:

train_path_img = "/content/drive/MyDrive/yolov8/data1/vehicle dataset/train/images"
train_path_label = "/content/drive/MyDrive/yolov8/data1/vehicle dataset/train/labels"
val_path_img = "/content/drive/MyDrive/yolov8/data1/vehicle dataset/valid/images"
val_path_label = "/content/drive/MyDrive/yolov8/data1/vehicle dataset/valid/labels"
test_path = "/content/yolo_data/test"

def train_test_split(path, neg_path=None, split=0.2):
    print("------ PROCESS STARTED -------")

    files = list(set([name[:-4] for name in os.listdir(path) if name.endswith('.jpg')]))

    print(f"--- This folder has a total number of {len(files)} images ---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    os.makedirs(train_path_img, exist_ok=True)
    os.makedirs(train_path_label, exist_ok=True)
    os.makedirs(val_path_img, exist_ok=True)
    os.makedirs(val_path_label, exist_ok=True)

    for filex in tqdm(files[:train_size]):
        if filex == 'classes':
            continue
        shutil.copy2(os.path.join(path, filex + '.jpg'), os.path.join(train_path_img, filex + '.jpg'))
        shutil.copy2(os.path.join(path, filex + '.txt'), os.path.join(train_path_label, filex + '.txt'))

    print(f"------ Training data created with 80% split ({len(files[:train_size])} images) ------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path) if name.endswith('.jpg')]))
        for filex in tqdm(neg_images):
            shutil.copy2(os.path.join(neg_path, filex + ".jpg"), os.path.join(train_path_img, filex + '.jpg'))

        print(f"------ Total {len(neg_images)} negative images added to the training data ------")
        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images ------")

    for filex in tqdm(files[train_size:]):
        if filex == 'classes':
            continue
        shutil.copy2(os.path.join(path, filex + '.jpg'), os.path.join(val_path_img, filex + '.jpg'))
        shutil.copy2(os.path.join(path, filex + '.txt'), os.path.join(val_path_label, filex + '.txt'))

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ------")
    print("------ TASK COMPLETED -------")

# Example usage:
# Call the function with the provided dataset and splitter image paths
train_test_split('/content/drive/MyDrive/yolov8/data')


------ PROCESS STARTED -------
--- This folder has a total number of 66 images ---


  0%|          | 0/53 [00:00<?, ?it/s]

------ Training data created with 80% split (53 images) ------


  0%|          | 0/13 [00:00<?, ?it/s]

------ Testing data created with a total of 13 images ------
------ TASK COMPLETED -------


In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.5 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.6/78.2 GB disk)


Model Training

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/yolov8/dataset.yaml epochs=10 imgsz=640 batch=8 project=/content/drive/MyDrive/yolov8/training_results name=vehicle


100% 21.5M/21.5M [00:00<00:00, 170MB/s]
Ultralytics YOLOv8.1.5 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/yolov8/dataset.yaml, epochs=10, time=None, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8/training_results, name=vehicle, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/yolov8/dataset.yaml epochs=10 imgsz=640 batch=8


# end

Inferencing

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/vehicle/weights/best.pt conf=0.55 source=/content/drive/MyDrive/yolov8/test_images

Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/2 /content/drive/MyDrive/yolov8/test_images/pexels-leonardo-hidalgo-12469410.jpg: 448x640 5 players, 65.2ms
image 2/2 /content/drive/MyDrive/yolov8/test_images/pexels-yahye-abdi-10512973.jpg: 448x640 3 players, 12.8ms
Speed: 3.5ms preprocess, 39.0ms inference, 286.3ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict3
💡 Learn more at https://docs.ultralytics.com/modes/predict


Copying the output files to selected direcctory

In [ ]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov8/output

Inferencing on video

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/football/weights/best.pt conf=0.77 source=/content/drive/MyDrive/yolov8/videos

Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

WARNING ⚠️ NMS time limit 0.550s exceeded
video 1/2 (1/258) /content/drive/MyDrive/yolov8/videos/pexels-tima-miroshnichenko-6079618 (2160p).mp4: 384x640 5 players, 77.2ms
video 1/2 (2/258) /content/drive/MyDrive/yolov8/videos/pexels-tima-miroshnichenko-6079618 (2160p).mp4: 384x640 5 players, 16.7ms
video 1/2 (3/258) /content/drive/MyDrive/yolov8/videos/pexels-tima-miroshnichenko-6079618 (2160p).mp4: 384x640 5 players, 19.6ms
video 1/2 (4/258) /content/drive/MyDrive/yolov8/videos/pexels-tima-miroshnichenko-6079618 (2160p).mp4: 384x640 5 players, 15.0ms
video 1/2 (5/258) /content/drive/MyDrive/yolov8/videos/pexels-tima-miroshnichenko-6079618 (2160p).mp4: 384x640 5 players, 24.1ms
video 1/2 (6/258) /content/drive/MyDrive/yolov8/videos/pexels-tima-miroshnichenko-6079618 (2160p).mp4: 384x640 5 players, 26.9ms
video 1/2 (7/2

In [ ]:
!cp -r /content/runs/detect/predict4 /content/drive/MyDrive/yolov8/output

# now